In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import time

In [2]:
# 定义超参数
batch_size = 32
learning_rate = 1e-3
num_epoches = 100

In [3]:
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
class Logstic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Logstic_Regression, self).__init__()
        self.logstic = nn.Linear(in_dim, n_class)

    def forward(self, x):
        out = self.logstic(x)
        return out

In [5]:

use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速


In [6]:
use_gpu

True

In [7]:
model = Logstic_Regression(28 * 28, 10)  # 图片大小是28x28

model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
model

Logstic_Regression (
  (logstic): Linear (784 -> 10)
)

In [9]:
for epoch in range(num_epoches):
    print('*' * 10)
    print('epoch {}'.format(epoch + 1))
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)  # 将图片展开成 28x28
        if use_gpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
            train_dataset))))
print('Time:{:.1f} s'.format(time.time() - since))

**********
epoch 1
[1/100] Loss: 2.157034, Acc: 0.346875
[1/100] Loss: 2.028956, Acc: 0.503073
[1/100] Loss: 1.915679, Acc: 0.578715
[1/100] Loss: 1.818451, Acc: 0.624349
[1/100] Loss: 1.734924, Acc: 0.654333
[1/100] Loss: 1.662632, Acc: 0.675816
Finish 1 epoch, Loss: 1.645766, Acc: 0.680667
**********
epoch 2
[2/100] Loss: 1.186911, Acc: 0.795729
[2/100] Loss: 1.154649, Acc: 0.803438
[2/100] Loss: 1.124006, Acc: 0.806354
[2/100] Loss: 1.091885, Acc: 0.811354
[2/100] Loss: 1.069460, Acc: 0.812438
[2/100] Loss: 1.047444, Acc: 0.813785
Finish 2 epoch, Loss: 1.042387, Acc: 0.814367
**********
epoch 3
[3/100] Loss: 0.888929, Acc: 0.828333
[3/100] Loss: 0.878783, Acc: 0.829323
[3/100] Loss: 0.867213, Acc: 0.831389
[3/100] Loss: 0.855118, Acc: 0.833125
[3/100] Loss: 0.841919, Acc: 0.834708
[3/100] Loss: 0.830333, Acc: 0.837066
Finish 3 epoch, Loss: 0.827293, Acc: 0.837383
**********
epoch 4
[4/100] Loss: 0.756148, Acc: 0.842292
[4/100] Loss: 0.746246, Acc: 0.845833
[4/100] Loss: 0.738354, Ac

In [10]:
model.eval()
eval_loss = 0.
eval_acc = 0.
for data in test_loader:
    img, label = data
    img = img.view(img.size(0), -1)
    if use_gpu:
        img = Variable(img, volatile=True).cuda()
        label = Variable(label, volatile=True).cuda()
    else:
        img = Variable(img, volatile=True)
        label = Variable(label, volatile=True)
    out = model(img)
    loss = criterion(out, label)
    eval_loss += loss.data[0] * label.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == label).sum()
    eval_acc += num_correct.data[0]
print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))
#print('Time:{:.1f} s'.format(time.time() - since))
print()

Test Loss: 0.305400, Acc: 0.916200

